# Integrators and sampling

In [1]:
# Preliminary imports
from simtk import openmm, unit
from simtk.openmm import app
import numpy as np

We'll the alanine dpeptide in vacuum imported from [`openmmtools`](http://openmmtools.readthedocs.io):

In [2]:
# Create an alanine dipeptide in vacuum
from openmmtools import testsystems
t = testsystems.AlanineDipeptideVacuum()
system, positions, topology = t.system, t.positions, t.topology

## Built-in OpenMM integrators

OpenMM provides a number of built-in [`Integrators`](http://docs.openmm.org/7.1.0/userguide/application.html#integrators) that may be useful for your intended application, but also provides a very flexible way to efficiently define new integrators that can still be executed very efficiently on the GPU.

All OpenMM integrators are derived from the [`Integrator`](http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.openmm.Integrator.html#simtk.openmm.openmm.Integrator) base class that provides some baseline functionality such that all integrators work the same way. We first have to bind an integrator to a `Context`. 

In [3]:
# Create a new integrator since the previously-created integrator was irrevocably bound to the previous Context
temperature = 298.0 * unit.kelvin
collision_rate = 91.0 / unit.picosecond
timestep = 2.0 * unit.femtoseconds
integrator = openmm.LangevinIntegrator(temperature, collision_rate, timestep)
# Create a Context for this integrator
context = openmm.Context(system, integrator)
# Set the positions
context.setPositions(positions)
# Minimize the potential energy
openmm.LocalEnergyMinimizer.minimize(context)

The initial velocities are, by default, zero, but we can select initial velocities from the Maxwell distribution with [`context.setVelocitiesToTemperature()`](http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.openmm.Context.html#simtk.openmm.openmm.Context.setVelocitiesToTemperature):

In [4]:
# Set velocities from Maxwell-Boltzmann distribution
context.setVelocitiesToTemperature(temperature)

To integrate a trajectory, we call [`integrator.step`](http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.openmm.Integrator.html#simtk.openmm.openmm.Integrator.step):

In [5]:
# Integrate some dynamics
nsteps = 100 # number of integrator steps
integrator.step(nsteps)

We can write a little loop to report some information every few timesteps:

In [6]:
# Run a few iterations of a few steps each, reporting potential energy
for iteration in range(10):
    integrator.step(100)
    state = context.getState(getEnergy=True)
    print('%8.3f ps : potential %12.6f kJ/mol' % (state.getTime() / unit.picoseconds, state.getPotentialEnergy() / unit.kilojoules_per_mole))

   0.400 ps : potential   -68.436951 kJ/mol
   0.600 ps : potential   -61.800964 kJ/mol
   0.800 ps : potential   -40.621155 kJ/mol
   1.000 ps : potential   -34.496613 kJ/mol
   1.200 ps : potential   -70.815399 kJ/mol
   1.400 ps : potential   -84.974030 kJ/mol
   1.600 ps : potential   -55.529419 kJ/mol
   1.800 ps : potential   -62.479706 kJ/mol
   2.000 ps : potential   -40.549622 kJ/mol
   2.200 ps : potential   -70.531708 kJ/mol


## The `Simulation` convenience class

While we could write our own wrapper to run a simulation and write data to disk, OpenMM's [`app` layer](http://docs.openmm.org/7.1.0/api-python/app.html) provides the [`Simulation` class](http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.app.simulation.Simulation.html#simtk.openmm.app.simulation.Simulation) to help you do this using a modular Python-based plugin architecture to specify which and how data should be stored. The [user guide](http://docs.openmm.org/7.1.0/userguide/application.html#a-first-example) provides a nice overview of this.

### Running a `Simulation` that writes data to the terminal

For example, to run a simulation that prints data to the terminal, we can use:

In [12]:
from sys import stdout
integrator = openmm.LangevinIntegrator(temperature, collision_rate, timestep)
simulation = app.Simulation(topology, system, integrator)
simulation.context.setPositions(positions)
simulation.minimizeEnergy()
simulation.reporters.append(app.StateDataReporter(stdout, 100, step=True, potentialEnergy=True, temperature=True))
simulation.step(1000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
100,-61.313568115234375,425.8075566049049
200,-65.38375854492188,279.1580486730683
300,-62.656982421875,265.4442317011469
400,-77.123046875,243.26907561476625
500,-56.972503662109375,273.6312775592466
600,-56.6046142578125,280.64286921973286
700,-69.30050659179688,263.5317931727405
800,-62.92010498046875,353.51756761749607
900,-56.76007080078125,303.6434881210641
1000,-50.563201904296875,409.50442908839824


### Running a `Simulation` that writes a trajectory

For example, to run a simulation that writes data in the extensible MDTraj HDF5 format, Amber NetCDF format, or CHARMM DCD format, we can use the [MDTraj Reporters](http://mdtraj.org/1.6.2/api/reporters.html):

In [19]:
import mdtraj
integrator = openmm.LangevinIntegrator(temperature, collision_rate, timestep)
simulation = app.Simulation(topology, system, integrator)
simulation.context.setPositions(positions)
simulation.minimizeEnergy()
reportInterval = 100
simulation.reporters.append(mdtraj.reporters.HDF5Reporter('output.h5', reportInterval, coordinates=True, time=True, cell=True, potentialEnergy=True, temperature=True))
simulation.reporters.append(mdtraj.reporters.DCDReporter('output.dcd', reportInterval))
simulation.reporters.append(mdtraj.reporters.NetCDFReporter('output.nc', reportInterval))
simulation.step(1000)

/Users/choderaj/miniconda3/lib/python3.5/site-packages/mdtraj/utils/validation.py:116: TypeCastPerformanceWarning: Casting xyz dtype=float64 to <class 'numpy.float32'> 
  TypeCastPerformanceWarning)
